In [3]:
# pip install mediapipe

In [1]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import math
from sklearn.decomposition import PCA
import pickle 

In [3]:
# from google.colab import drive
# drive.mount('/content/AFLW2000')

In [4]:
with open('pitch_model.pkl', 'rb') as f:
    pitch_model = pickle.load(f)

In [5]:
with open('yaw_model.pkl', 'rb') as f:
    yaw_model = pickle.load(f)

In [6]:
with open('roll_model.pkl', 'rb') as f:
    roll_model = pickle.load(f)

In [7]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2


    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

In [8]:
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 
mp_drawing_styles = mp.solutions.drawing_styles

In [11]:
import cv2
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
    
        results = holistic.process(image)
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        
        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).reshape(1, -1))
            
            pitch = pitch_model.predict(face_row)
            yaw = yaw_model.predict(face_row)
            roll = roll_model.predict(face_row)
            
            cv2.putText(image, f"Pitch: {pitch}", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 0, 0), 2, cv2.LINE_AA, False)
            cv2.putText(image, f"Yaw: {yaw}", (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 0, 0), 2, cv2.LINE_AA, False)
            cv2.putText(image, f"Roll: {roll}", (0, 90), cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 0, 0), 2, cv2.LINE_AA, False)

            yaw = -yaw
            tdx = NoseX
            tdy = NoseY
            size=100
            x1 = size * (cos(yaw) * cos(roll)) + tdx
            y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy
            x2 = size * (-cos(yaw) * sin(roll)) + tdx
            y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

            x3 = size * (sin(yaw)) + tdx
            y3 = size * (-cos(yaw) * sin(pitch)) + tdy
            cv2.line(image, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)
        except:
            pass

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [15]:
cap = cv2.VideoCapture('Video.mp4')
ret, frame = cap.read()  
h, w, _ = frame.shape 
frameTime = 2

fourcc = cv2.VideoWriter_fourcc(*"XVID") 
fps = 1000 / frameTime 
writer = cv2.VideoWriter("Pothole testing 2.mp4", fourcc, fps, (w, h)) 

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while ret: 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        results = holistic.process(image)
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        
        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).reshape(1, -1))
            
            pitch = pitch_model.predict(face_row)
            yaw = yaw_model.predict(face_row)
            roll = roll_model.predict(face_row)
            
            cv2.putText(image, f"Pitch: {pitch}", (0, 90), cv2.FONT_HERSHEY_SIMPLEX, 1,  (255, 0, 0), 2, cv2.LINE_AA, False)
            cv2.putText(image, f"Yaw: {yaw}", (0, 120), cv2.FONT_HERSHEY_SIMPLEX, 1,  (255, 0, 0), 2, cv2.LINE_AA, False)
            cv2.putText(image, f"Roll: {roll}", (0, 150), cv2.FONT_HERSHEY_SIMPLEX, 1,  (255, 0, 0), 2, cv2.LINE_AA, False)
            
            draw_axis(image,pitch,yaw,roll,NoseX, NoseY)
        except:
            pass
        writer.write(frame)
        cv2.imshow('Video', image)

        if cv2.waitKey(frameTime) & 0xFF == ord('q'):
            break
        ret, frame = cap.read()

writer.release()
cap.release()
cv2.destroyAllWindows()